In [ ]:
import reflexgenerator.sources
from reflexgenerator.sources import (
    Register, BitMask, Bit, IOElement, Metadata, Collection)
from reflexgenerator.sources.io import load

from pathlib import Path
import pandas as pd
from reflexgenerator.generator.markdown import AnchorReference


In [ ]:
schema = r"schema\device.json"
fw_target = r"schema\example.yml"


In [ ]:
jsondata = load(fw_target)


In [ ]:
metadata = Metadata(
    device=jsondata["device"],
    whoAmI=jsondata["whoAmI"],
    architecture=jsondata["architecture"],
    firmwareVersion=jsondata["firmwareVersion"],
    hardwareTargets=jsondata["hardwareTargets"])
print(metadata)

In [ ]:
# build masks
masks = Collection([BitMask(**mask) for mask in jsondata["masks"]])
masks_df = pd.DataFrame([mask.to_dict() for mask in masks])
masks_df.head(5)

In [ ]:
# build registers
regs = Collection([Register(**reg) for reg in jsondata["registers"]])
register_df = pd.DataFrame([reg.to_dict() for reg in regs])
register_df.head(5)



In [ ]:
ios = Collection([IOElement(**io) for io in jsondata["ios"]])
ios_df = pd.DataFrame([io.to_dict() for io in ios])
ios_df.head(5)

In [ ]:
for i in register_df.index:
    if register_df.at[i, 'maskType']:
        if isinstance(register_df.at[i, 'maskType'][0], BitMask):
            register_df.at[i, 'maskType'] = [x.render_pointer() for x in register_df.at[i, 'maskType']]



In [ ]:
masks_df["name"] = masks_df["name"].apply(lambda name: AnchorReference.REFERENCES[name].render_reference())

In [ ]:
txt = f"""# Registers

{register_df.to_markdown()}


# Masks

{masks_df.to_markdown()}

# IOs

{ios_df.to_markdown()}
"""

with open("documentation.md", "w") as text_file:
    text_file.write(txt)

import markdown
with open("documentation.html", "w") as text_file:
    text_file.write(markdown.markdown(txt))
